In [59]:
import requests
from bs4 import BeautifulSoup
import json

In [60]:
#fonction qui recupere les tags avec comme attribut data-ar-restaurant-id
def restaurants_id (tag):
    return tag.has_attr('data-ar-restaurant-id')

In [61]:
#fonction qui recupere les tags avec comme attribut data-product-id
def plats_id (tag):
    return tag.has_attr('data-product-id')

In [62]:
#Fonction qui recupere les plats d'un restaurant donné et les ajoute à une liste
def recuperer_liste_plats(link, id_restaurant, liste_plats):
    req_carte = requests.get(link)

    page_carte = req_carte.text
    soup_carte = BeautifulSoup(page_carte, 'html.parser')

    plats = soup_carte.find_all(plats_id)
    
    for plat in plats:
        soup_p = BeautifulSoup(str(plat) , 'html.parser')

        #id
        id_plat = soup_p.find("li")["data-product-id"]

        #On vérifie que le plat ajouté n'est pas deja dans la liste
        isInList = False
        for p in liste_plats:
            if p["id"]==id_plat:
                isInList = True

        if not isInList:
            #id_restaurant
            id_restaurant = 0

            #nom du plat
            name = soup_p.find("h4" , {"class" : "name"}).find("span").text

            #logo
            logo = soup_r.find("img" )['src']

            #prix
            prix = soup_p.find("div" , {"class" : "price"}).text
            prix = " ".join(prix.split())

            #description
            description = soup_p.find("div" , {"class" : "description"}).text

            objet = {
                "id_restaurant" : id_restaurant,
                "id" : id_plat,
                "name" : name,
                "prix" : prix,
                "descrption" : description,
                "logo" : logo
            }
            #print(objet["prix"])

            liste_plats.append(objet)

In [63]:
#Fonction qui recupere les donnees des restaurants sur une page alloresto et les ajoute à une liste
def recuperer_liste_restaurants(link, liste_restaurants, liste_plats):
    req = requests.get(link)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    restaurants = soup.find_all(restaurants_id)
    i=0

    for restaurant in restaurants:
        restaurant = str(restaurant) 
        soup_r = BeautifulSoup(restaurant, 'html.parser')

        #link
        link = soup_r.find("h5" , {"class" : "name"}).find("a")["href"]

        #On vérifie que le restaurant n'est pas déjà dans la liste
        isInList = False
        for r in liste_restaurants:
            if r["link"]==link:
                isInList = True

        if not isInList:

            #id
            id_restaurant = i
            i = i + 1

            #name
            name = soup_r.find("h5" , {"class" : "name"}).text.replace("\n","")

            #logo
            logo = soup_r.find("img" )['src']

            #address
            address = soup_r.find("div" , {"class" : "address"}).text
            address = " ".join(address.split())

            #specialties
            specialties = [x.text for x in soup_r.find("div" , {"class" : "cuisines"}).find_all("li")]

            #work time 
            #TODO: A RETRAVAILLER
            work_time = None

            #minimum_order
            temp = soup_r.find("li", {"class" : "minimum-order"})
            if temp is not None:
                minimum_order = temp.find("strong").text
                minimum_order = " ".join(minimum_order.split())
            else : 
                minimum_order = "0€"

            #print(type(name))
            objet = {
                "id" : id_restaurant,
                "name" : name,
                "logo" : logo,
                "address" : address,
                "specialties" : specialties,
                "work_time" : work_time,
                "minimum_order" : minimum_order,
                "link" : link
            }

            liste_restaurants.append(objet)
            #Appel de la fonction ajoutant la liste des plats
            recuperer_liste_plats(link, id_restaurant, liste_plats)

    #print(liste_restaurants)

In [64]:
#Liste des restaurants
liste_restaurants = []
liste_plats = []
link = "https://www.alloresto.fr/restaurants-livraison-a-domicile/zone-livraison/paris-12"

recuperer_liste_restaurants(link, liste_restaurants, liste_plats)


In [65]:
liste_restaurants

[{'address': '11 Rue Emile Zola, 93100 MONTREUIL',
  'id': 1,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/okinawa-93100/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/5313.png',
  'minimum_order': '20.00€',
  'name': 'Okinawa',
  'specialties': ['Japonais'],
  'work_time': None},
 {'address': '14 Rue de Nice, 75011 PARIS 11',
  'id': 1,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/ete-edo-750110/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/132312.png',
  'minimum_order': '18.00€',
  'name': 'Eté Edo',
  'specialties': ['Japonais', 'Chinois'],
  'work_time': None},
 {'address': '25 Rue du Docteur Heulin, 75017 PARIS 17',
  'id': 1,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/dabbawalas-paris75017/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/9849dabwalas200.png',
  'minimum_order': '18.0

In [13]:
#TODO : appui sur le bouton pour avoir tous les restaurants

In [52]:
#test = json.dumps(liste_plats)
#test

In [4]:
#Tests Selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

# Create a new instance of the Firefox driver
driver = webdriver.Firefox()
#driver = webdriver.PhantomJS()
#driver = webdriver.Remote("http://localhost:4444/wd/hub", webdriver.DesiredCapabilities.HTMLUNIT.copy())


# go to the google home page
driver.get("http://www.google.com")

# the page is ajaxy so the title is originally this:
print (driver.title)

# find the element that's name attribute is q (the google search box)
inputElement = driver.find_element_by_name("q")

# type in the search
inputElement.send_keys("cheese!")

# submit the form (although google automatically searches now without submitting)
inputElement.submit()

try:
    # we have to wait for the page to refresh, the last thing that seems to be updated is the title
    WebDriverWait(driver, 10).until(EC.title_contains("cheese!"))

    # You should see "cheese! - Google Search"
    print(driver.title)

finally:
    driver.quit()

Google
cheese! - Recherche Google
